In [ ]:
import pandas as pd

cash = pd.read_csv("project_dataset/extract - cash request - data analyst.csv")
print(cash.info())

<class 'pandas.DataFrame'>
RangeIndex: 23970 entries, 0 to 23969
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          23970 non-null  int64  
 1   amount                      23970 non-null  float64
 2   status                      23970 non-null  str    
 3   created_at                  23970 non-null  str    
 4   updated_at                  23970 non-null  str    
 5   user_id                     21867 non-null  float64
 6   moderated_at                16035 non-null  str    
 7   deleted_account_id          2104 non-null   float64
 8   reimbursement_date          23970 non-null  str    
 9   cash_request_received_date  16289 non-null  str    
 10  money_back_date             16543 non-null  str    
 11  transfer_type               23970 non-null  str    
 12  send_at                     16641 non-null  str    
 13  recovery_status             3330 non-null 

In [4]:
fees = pd.read_csv("project_dataset/extract - fees - data analyst - .csv")
print(fees.info())

<class 'pandas.DataFrame'>
RangeIndex: 21061 entries, 0 to 21060
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               21061 non-null  int64  
 1   cash_request_id  21057 non-null  float64
 2   type             21061 non-null  str    
 3   status           21061 non-null  str    
 4   category         2196 non-null   str    
 5   total_amount     21061 non-null  float64
 6   reason           21061 non-null  str    
 7   created_at       21061 non-null  str    
 8   updated_at       21061 non-null  str    
 9   paid_at          15531 non-null  str    
 10  from_date        7766 non-null   str    
 11  to_date          7766 non-null   str    
 12  charge_moment    21061 non-null  str    
dtypes: float64(2), int64(1), str(10)
memory usage: 2.1 MB
None


In [6]:
fees["cash_request_id"] = fees["cash_request_id"].astype("Int64")

fees_date = ["created_at", "updated_at", "paid_at", "from_date", "to_date"]


for col in fees_date:
    fees[col] = pd.to_datetime(fees[col], errors="coerce")

print(fees.info())
print(fees.head())    



lexique = pd.read_excel("project_dataset/Lexique - Data Analyst.xlsx")
print(lexique.info())

<class 'pandas.DataFrame'>
RangeIndex: 21061 entries, 0 to 21060
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   id               21061 non-null  int64              
 1   cash_request_id  21057 non-null  Int64              
 2   type             21061 non-null  str                
 3   status           21061 non-null  str                
 4   category         2196 non-null   str                
 5   total_amount     21061 non-null  float64            
 6   reason           21061 non-null  str                
 7   created_at       21061 non-null  datetime64[us, UTC]
 8   updated_at       21061 non-null  datetime64[us, UTC]
 9   paid_at          15438 non-null  datetime64[us, UTC]
 10  from_date        6749 non-null   datetime64[us, UTC]
 11  to_date          6512 non-null   datetime64[us, UTC]
 12  charge_moment    21061 non-null  str                
dtypes: Int64(1), datetime64[us,